# Import modules

In [1]:
import os
import cv2
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler

import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

from matplotlib import pyplot as plt
from dataloader import *

In [2]:
TRAIN_DIR = '../dataset/train/'
TEST_DIR = '../dataset/test/'

# Check hardware accelerator
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
### Exploring Dataset

classes = os.listdir(TRAIN_DIR)
print("Total Classes: ",len(classes))

train_count = 0
test_count = 0

for _class in classes:
    train_count += len(os.listdir(TRAIN_DIR + _class))
    test_count += len(os.listdir(TEST_DIR + _class))
    
print("Total train images: ",train_count)
print("Total test images: ",test_count)

Total Classes:  4
Total train images:  7330
Total test images:  34


In [4]:
# Utility to apply transforms
def get_transform():
  mean = (127.5)
  std = (127.5)
  normalize = T.Normalize(mean=mean, std=std)
  return T.Compose([normalize])

# Loading Classification Dataset

In [5]:
train_dataset = CustomDataset(TRAIN_DIR, transforms=get_transform())
test_dataset = CustomDataset(TEST_DIR, transforms=get_transform())

print(len(train_dataset))
print(len(test_dataset))

7330
34


In [8]:
train_data_loader = DataLoader(dataset = train_dataset, batch_size = 128, shuffle=True)
test_data_loader = DataLoader(dataset = test_dataset, batch_size = 128, shuffle=False)

In [9]:
print(len(train_data_loader))

46


In [12]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  return torch.device(dev)

In [13]:
name = 'models'

try:
    os.makedirs(os.path.join(os.getcwd(), f'{name}'))
except FileExistsError:
    print("Directory already exists!")
    pass

modelDir = os.path.join(os.getcwd(), f'{name}')

Directory already exists!


# Define Training Loop

In [14]:
# Training function
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs):
  device = set_device()
  train_loss = []
  train_acc = []

  for epoch in range(num_epochs):
    print("Epoch number {}".format(epoch + 1))
    start = time.time()
    model.train()
    running_loss = 0.0
    running_correct = 0
    total = 0
      
    # Training
    for data in train_data_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)
      
      #Reset Grads
      optimizer.zero_grad()
      
      #Forward ->
      outputs = model(images)

      # pred
      _, predicted = torch.max(outputs.data, 1)

      
      #Calculate Loss & Backward, Update Weights (Step)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() 
      running_correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_data_loader)
    epoch_acc = 100.00 * running_correct / total

    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)

    print("  - Training dataset: Got %d out of %d images correctly (%.3f%%). \nEpoch loss: %.3f"
        % (running_correct, total, epoch_acc, epoch_loss))

    test_acc = evaluate_model(model, test_loader)
      
    end = time.time()

    print("-  Epoch Time : {} \n".format(int(end-start)))

  print('Finished')
  return model, train_acc, train_loss, test_acc

In [15]:
# Testing function
def evaluate_model(model, test_loader):
  model.eval()
  predicted_correctly_on_epoch = 0
  total = 0
  best_acc = 0.0
  acc = []
  device = set_device()

  with torch.no_grad():
    for data in test_data_loader: 
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)

      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)

      predicted_correctly_on_epoch += (predicted == labels).sum().item()
  
  epoch_acc = 100.0 * predicted_correctly_on_epoch / total
  acc.append(epoch_acc)

  if epoch_acc > best_acc:
    best_acc = epoch_acc
    torch.save(model.state_dict(), os.path.join(modelDir, 'best_model.pth'))

  print("  - Testing dataset: Got %d out of %d images correctly (%.3f%%)"
        % (predicted_correctly_on_epoch, total, epoch_acc))
  
  return acc

# Define the model

In [16]:
# # for resnet
# model = models.resnet18(pretrained=True)
# num_features = model.fc.in_features
# num_classes = 4
# model.fc = nn.Linear(num_features, num_classes)
# print(model)

# for mobilenet
model = models.mobilenet_v2(pretrained=True)
num_features = model.classifier[1].in_features
num_classes = 4
model.classifier[1] = nn.Linear(num_features, num_classes)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Define Hyperparameters

In [17]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
EPOCHS = 20

# Training phase

In [18]:
model_trained, TRAIN_LOSS, TRAIN_ACC, TEST_ACC = train_model(model, train_data_loader, test_data_loader, criterion, optimizer, lr_scheduler, num_epochs=EPOCHS)

Epoch number 1
  - Training dataset: Got 4813 out of 7330 images correctly (65.662%). 
Epoch loss: 0.777
  - Testing dataset: Got 16 out of 34 images correctly (47.059%)
-  Epoch Time : 18 

Epoch number 2
  - Training dataset: Got 6644 out of 7330 images correctly (90.641%). 
Epoch loss: 0.277
  - Testing dataset: Got 32 out of 34 images correctly (94.118%)
-  Epoch Time : 18 

Epoch number 3
  - Training dataset: Got 7210 out of 7330 images correctly (98.363%). 
Epoch loss: 0.078
  - Testing dataset: Got 32 out of 34 images correctly (94.118%)
-  Epoch Time : 18 

Epoch number 4
  - Training dataset: Got 7289 out of 7330 images correctly (99.441%). 
Epoch loss: 0.032
  - Testing dataset: Got 33 out of 34 images correctly (97.059%)
-  Epoch Time : 17 

Epoch number 5
  - Training dataset: Got 7326 out of 7330 images correctly (99.945%). 
Epoch loss: 0.012
  - Testing dataset: Got 33 out of 34 images correctly (97.059%)
-  Epoch Time : 18 

Epoch number 6
  - Training dataset: Got 7329

KeyboardInterrupt: 

In [ ]:
# Save the final model as well
torch.save(model.state_dict(), os.path.join(modelDir, 'model.pth'))